In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [57]:
from statistics import stdev, mean, median
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import numpy as np
import random
import scipy
from math import sqrt
import copy
import csv

In [139]:
def import_csv():
        """Метод считывает данные из CSV файла и преобразует их в DataFrame."""
        try:
            with open(input('Введите путь к файлу: \n'), encoding='utf-8-sig') as data_file:
                csv = []
                A = []
                name_index = []
                for line in data_file:
                    csv = line.strip().split(';')
                    name_index.append(csv.pop(0))
                    for n, x in enumerate(csv):
                        if ',' in x:
                            csv[n] = float(x.replace(',', '.'))
                        else:
                            try:
                                csv[n] = float(x)
                            except:
                                pass
                    A.append(csv)
                
                name_column_index = name_index.pop(0)
                name_columns = A[0]
                l_data = A[1:]
                A = pd.DataFrame(A[1:], 
                                 columns=name_columns,
                                 index=name_index)
                A.index.name = name_column_index
            return A, l_data, name_index, name_columns
        except FileNotFoundError:
            print('Искомый файл не найден! Попробуйте еще раз: \n')
            return import_csv()


def input_way():
    """Метод предназначен для защиты от неверного ввода номера решаемой задачи."""
    try:
        way = int(input('\nКакую задачу решать? (1, 2 или 3): '))
        if way not in (1, 2, 3):
            print('Введите цифру 1, 2 или 3!')
            return input_way()
        else:
            return way
    except:
        print('Номер решаемой задачи должен быть числом!')
        return input_way()


def input_number():
    """Метод предназначен для защиты от неверного ввода количества ранговых оценок."""
    try:
        way = int(input('\nКоличество экспертов: '))
        return way
    except:
        print('Номер решаемой задачи должен быть числом!')
        return input_number()


def input_csv_rank(number):
    """Метод формирует массив с бинарными отношениями по всем экспертам."""
    list_data = []
    for n in range(number):
        data, l_data, name_index, name_columns = import_csv()
        list_data.append(l_data)
    return list_data, name_index, name_columns
    


def expert_opinions(data, name_index):
    """Математическая обработка экспертных оценок
    
    Метод рассчитывает математическое ожидание, дисперсию, 
    среднеквадратическое отклонение и доверительный интервал
    по выборкам, состоящим из экспертных оценок по каждой позиции.
    
    """
    mean = []
    dispersion = []
    standard_deviation = []
    confidence_interval = []
    for i in data.index:
        mean.append(round(data.loc[i].mean(), 4))
        disp = round(np.var(np.array(data.loc[i])) * len(data.loc[i]) / (len(data.loc[i]) - 1), 4)
        dispersion.append(disp)
        standard_deviation.append(round(sqrt(disp), 4))
        confidence_interval.append(str([round(data.loc[i].mean() - 2*sqrt(disp), 4), 
                                        round(data.loc[i].mean() + 2*sqrt(disp), 4)]))
    
    data_expert_opinions = np.array([mean, dispersion, standard_deviation, confidence_interval])
    data_expert_opinions = pd.DataFrame(data_expert_opinions.transpose(),
                                       columns=['Мат.ожидание', 'Дисперсия', 'Стандарт.откл', 'Доверит.интерв (+-2𝜎)'],
                                       index=name_index)
    return data_expert_opinions


def rank_estimates(data, name_index, name_columns):
    """Математическая обработка ранговых оценок
    
    Метод рассчитывает медианы рангов, массив коэффициентов компетентности экспертов
    и определяет средневзвешенные ранги с учетом коэффициентов компетентности.
    
    """
    mean = []
    median = []
    for i in data.index:
        median.append(data.loc[i].median())
        mean.append(round(data.loc[i].mean(), 4))
    
    dispersion = []
    for i in data:
        disp = 0
        count = 1 / (len(data[i]) - 1)
        for num, j in enumerate(data.index):
            disp += (data[i][j] - mean[num])**2
        dispersion.append(1 / (count * disp))
    
    competence = []
    competence_str = []
    for i in dispersion:
        competence.append(round((i / sum(dispersion)), 4))
        competence_str.append(f'{round(100 * (i / sum(dispersion)), 1)}%')
    
    for i in ('Медиана', 'Средневзв.ранг'):
        competence_str.append(' ')
        name_columns.append(i)
    
    competence_df = pd.DataFrame([competence_str], 
                                 columns=name_columns, 
                                 index=['Коэф.комп-ти'])
    
    rank_with_competence = []
    for i in data.index:
        rank = 0
        for num, j in enumerate(data):
            rank += data[j][i] * competence[num]
        rank_with_competence.append(round(rank, 2))
    
    data['Медиана'] = median
    data['Средневзв.ранг'] = rank_with_competence
    data = data.append(competence_df)
    print(data)


def Kemeny_median(number):
    """Функция рассчитывает медиану Кемени"""
    list_data, name_index, name_columns = input_csv_rank(number)
    
    matrix = np.zeros((len(list_data[1]), len(list_data[1])))
    for i in range(len(matrix)):
        for j in range(i + 1):
            matrix[i][j] = 1
    
    def fun(list_data, matrix):
        """Целевая функция"""
        sum_rank = []
        for i in list_data:
            sum_matrix_and_list_data = np.matrix(i) - np.matrix(matrix)
            sum_ = sum(list(sum(map(lambda x: abs(x), np.array(sum_matrix_and_list_data)))))
            sum_rank.append(sum_)
        return sum(sum_rank)
    
    
    fun(list_data)
    
    
    


way = input_way()

if way == 1:
    data, list_data, name_index, name_columns = import_csv()
    print('\nВы импортировали: \n', data)
    data_expert_opinions = expert_opinions(data, name_index)
    print('\n\n', 'Математическая обработка экспертных оценок'.center(50), '\n')
    print(data_expert_opinions)

elif way == 2:
    data, list_data, name_index, name_columns = import_csv()
    print('\nВы импортировали: \n', data)
    print('\n\n', 'Математическая обработка ранговых оценок'.center(50), '\n')
    rank_estimates(data, name_index, name_columns)

else:
    print('\n\n', 'Математическая обработка бинарных отношений'.center(50), '\n')
    number = input_number()
    
    Kemeny_median(number)


Какую задачу решать? (1, 2 или 3): 3


    Математическая обработка бинарных отношений     


Количество экспертов: 6
Введите путь к файлу: 
/Users/evgeniakiseleva/Desktop/1.csv
Введите путь к файлу: 
/Users/evgeniakiseleva/Desktop/2.csv
Введите путь к файлу: 
/Users/evgeniakiseleva/Desktop/3.csv
Введите путь к файлу: 
/Users/evgeniakiseleva/Desktop/4.csv
Введите путь к файлу: 
/Users/evgeniakiseleva/Desktop/5.csv
Введите путь к файлу: 
/Users/evgeniakiseleva/Desktop/6.csv

list_data  [[[1. 0. 0. 0. 0.]
  [1. 1. 1. 0. 0.]
  [1. 0. 1. 1. 0.]
  [1. 1. 0. 1. 0.]
  [1. 1. 1. 1. 1.]]

 [[1. 0. 1. 0. 0.]
  [1. 1. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [1. 1. 1. 1. 0.]
  [1. 1. 1. 1. 1.]]

 [[1. 0. 1. 0. 0.]
  [1. 1. 1. 1. 0.]
  [0. 0. 1. 1. 0.]
  [1. 0. 0. 1. 0.]
  [1. 1. 1. 1. 1.]]

 [[1. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1.]
  [1. 0. 1. 1. 0.]
  [1. 0. 0. 1. 0.]
  [1. 0. 1. 1. 1.]]

 [[1. 0. 0. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]
  [1.

In [ ]:
/Users/evgeniakiseleva/Desktop/1.csv

In [ ]:
/Users/evgeniakiseleva/Desktop/tt7_1.csv
/Users/evgeniakiseleva/Desktop/tt7_2.csv
/Users/evgeniakiseleva/Desktop/tt7_3.csv

In [132]:
import numpy as p 

matA = p.matrix([[-11, -22], [-33, -44]]) 
matB = p.matrix([[2,2], [2,2]]) 
print("Matrix A:\n", matA) 
print("Matrix B:\n", matB) 
print("Subtraction of Matrix A and Matrix B:\n",(matA - matB))

a = sum(list(sum(map(lambda x: abs(x), np.array(matA)))))
#a = sum(sum(list(map(lambda x: abs(x), matA))))
#a = sum(sum(map(lambda x: abs(x), matA)))
print(a)


Matrix A:
 [[-11 -22]
 [-33 -44]]
Matrix B:
 [[2 2]
 [2 2]]
Subtraction of Matrix A and Matrix B:
 [[-13 -24]
 [-35 -46]]
110
